# HDS5210-2022 Midterm

In the midterm, you're going to use all of the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged: "home" if they are below 30% risk and the recommended hospital if they are at or above 30%.

Each step in the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure that you're getting those steps correct.

The midterm is due at 11:59 PM CST on Monday, March 14th.

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should raise a ValueException that includes a message with the invalid input and which parameter that input was provided as.

NOTES:
1. In the final step there is a rule to convert form raw score to percentile.  In that conversion, 17-25 maps to 59-88% and ≥26 maps to >99%.  For our purposes, we want these to be specific % number outputs.  Use the following rule:
 * If score is between 17 and 25, percentile should be 0.59
 * If score is greater than or equal to 26, percentile should be 0.99


In [393]:
def priest(sex, age, breath, o2sat, heart, systolicbp, temp, alertness, inspired, status):
    
    """
    This function uses the logic from https://www.mdcalc.com/priest-covid-19-clinical-severity-score#use-cases
    to compute the priest score for this patient.
    
    >>> priest(male, 52, 8,0.94,60,100,37, "Confused or not alert", "Supplemental oxygen", "Limited self-care")
    17
    
    >>> priest(female, 52, 8,0.94,60,100,37, "Confused or not alert", "Supplemental oxygen", "Limited self-care")
    16
    
    """
    
    score = 0
    if sex.lower() == "male":
        score += 1
        
    if age >=50 and age <=65:
        score += 2
    elif age >=66 and age <=80: 
        score += 3
    elif age > 80:
        score += 4
    
    if breath < 9:
        score += 3
    elif breath >= 9 and breath <= 11:
        score += 1
    elif breath >= 21 and breath <= 24:
        score += 2
    elif breath > 24:
        score += 3
        
    if o2sat >= 0.94 and o2sat <= 0.95:
        score += 1
    elif o2sat >= 0.92 and o2sat <= 0.93:
        score += 2
    elif o2sat <= 0.92:
        score += 3
        
    if heart < 41:
        score += 3
    elif heart >= 41 and heart <= 50:
        score += 1
    elif heart >= 91 and heart <= 110:
        score += 1
    elif heart >= 111 and heart <= 130:
        score += 2
    elif heart >= 130:
        score += 3
        
    if systolicbp < 91:
        score += 3
    elif systolicbp >= 91 and systolicbp <= 100:
        score +=2
    elif systolicbp >= 101 and systolicbp <= 110:
        score +=1
    elif systolicbp >= 219:
        score +=3
        
    if temp < 35.1:
        score += 3
    elif temp >= 25.1 and temp <= 36:
        score += 1
    elif temp >= 38.1 and temp <= 39:
        score += 1
    elif temp > 39:
        score += 2
        
    if alertness.lower() == "confused or not alter":
        score += 3
    
    if inspired.lower() == "supplemental oxygen":
        score += 2
        
    if status.lower() == "limited strenuous activity, can do light activity":
        score += 1
    elif status.lower() == "limited activity, can self-care":
        score += 2
    elif status.lower() == "limited self-care":
        score += 3
    elif status.lower() == "bed/chari bound, no self-care":
        score += 4
    risk = []
    if score == 0 or score == 1:
        pct = 0.01
    elif score == 2 or score == 3:
        risk = 0.02
    elif score == 4:
        risk = 0.03
    elif score == 5:
        risk = 0.09
    elif score == 6:
        risk = 0.15
    elif score == 7:
        risk = 0.18
    elif score == 8:
        risk = 0.22
    elif score == 9:
        risk = 0.26
    elif score == 10:
        risk = 0.29
    elif score == 11:
        risk = 0.34
    elif score == 12:
        risk = 0.48
    elif score == 13:
        risk = 0.46
    elif score == 14:
        risk = 0.47
    elif score == 15:
        risk = 0.49
    elif score == 16:
        risk = 0.55
    elif score >= 17 and score <= 25:
        risk = 0.59
    elif score >= 26:
        risk = 0.99

    return risk
        

## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk percetage of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None without raising any exceptions
5. Include a good docstring with at least five test cases.


In [445]:
import requests
import json

def find_hospital(age, sex, risk):
    
    """
    >>> find_hospital(42, "female", 0.2)
    Select Specialty Hospital - Northeast Atlanta
    
    >>> find_hospital(45, "male", 0.5)
    Emory Dunwoody Medical Center
    
    >>> find_hospital(46, "female", 0.2)
    Select Specialty Hospital - Northeast Atlanta
    
    >>> find_hospital(54, "female", 0.5)
    Emory Dunwoody Medical Center
    
    >>> find_hospital(57, "male", 0.7)
    Emory Dunwoody Medical Center
    
    
    """
    
    query = {"age": age, "sex":sex, "risk_pct": risk}
    response = requests.get('https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1', params=query)
    data = response.json()
    return(data.get('hospital'))

find_hospital(42, "female", 0.2)   

'Select Specialty Hospital - Northeast Atlanta'

## Part 3: Get the address for that hospital from a webpage

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list on this webpage to lookup the address for that hospital, based on its name.

https://www.officialusa.com/stateguides/health/hospitals/georgia.html

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data on the webpage above to find the addres for that hospital.
2. I've said that all the hospitals are in Atlanta, but this webpage has hospitals from all of Georgia.  So, make sure you verify that the row of data you're using is in Atlanta, just in case there are hospitals with the same name in different cities.
3. Your code will have to parse the HTML on the webpage and turn that into some kind of data structure that you can search through to find the right hospital.
4. If you do find more than one matching hospital in Atlanta with the same name, you should raise a KeyError.
5. If the hospital name isn't found, the function should raise a KeyError.
6. Be sure to use good docstring documentation and includes at least 3 test cases.

In [472]:
import pandas as pd
import requests
import numpy as np
url = 'https://www.officialusa.com/stateguides/health/hospitals/georgia.html'
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

dfs = pd.read_html(r.text)
nd = dfs[0]
atl = nd[nd['City'] == "ATLANTA"]

def get_address(hospital_name):
    for i in range(len(nd)):
        if hospital_name != nd['Hospital Name'][i]:
            i +=1
        else:
            print(nd['ADDRESS'][i])

get_address("SELECT SPECIALTY HOSPITAL - ATLANTA")

550 PEACHTREE STREET, NE


## Part 4: Run the risk calculator on a population

The `/data` folder has a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.

In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes a file name as a parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you below.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [482]:
import csv
import pandas as pd

def process_people(file_name):
    df = pd.read_csv(file, header='infer', sep="|")
    sex = df['sex']
    age = df['age']
    breath = df['breath']
    o2sat = df['o2 sat']
    heart = df['heart']
    systolicbp = df['systolic bp']
    temp = df['temp']
    alertness = df['alertness']
    inspired = df['inspired']
    status = df['status']

    risk = []
    for i in range(len(df)):
        risk.append(priest(sex[i], age[i], breath[i], o2sat[i], heart[i], systolicbp[i], temp[i], alertness[i], inspired[i], status[i]))
    df['risk'] = risk
    df


    hosp = []
    for i in range(len(df)):
        hosp.append(find_hospital(age[i], sex[i], risk[i]))
    df['Hospital Name'] = hosp

    df['Hospital Name'] = df['Hospital Name'].str.lower()
    atl['Hospital Name'] = atl['Hospital Name'].str.lower()
    ndf = pd.merge(atl, df, on = 'Hospital Name', how='inner')
    print(dict(ndf))

file = "/data/people.psv"
process_people(file)

{'City': 0     ATLANTA
1     ATLANTA
2     ATLANTA
3     ATLANTA
4     ATLANTA
       ...   
94    ATLANTA
95    ATLANTA
96    ATLANTA
97    ATLANTA
98    ATLANTA
Name: City, Length: 99, dtype: object, 'Hospital Name': 0                        emory dunwoody medical center
1     childrens healthcare of atlanta at scottish rite
2     childrens healthcare of atlanta at scottish rite
3     childrens healthcare of atlanta at scottish rite
4        select specialty hospital - northeast atlanta
                            ...                       
94                     wesley woods geriatric hospital
95                     wesley woods geriatric hospital
96                     wesley woods geriatric hospital
97                     wesley woods geriatric hospital
98                     wesley woods geriatric hospital
Name: Hospital Name, Length: 99, dtype: object, 'TYPE': 0     GENERAL ACUTE CARE
1               CHILDREN
2               CHILDREN
3               CHILDREN
4         LONG TERM 

/opt/tljh/user/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` in the `/data` folder.  Write some code to check your results.  This does not need to be a function.

In [478]:
file = "/data/people_results.json"
process_people(file)

KeyError: 'sex'

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

---

In [1]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git add "midterm-2022.ipynb"
    !git commit -a -m "Submitting the midterm"
    !git push
else:
    print('''
    
OK. We can wait.
''')


Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

 yes


[main d88bc5b] Submitting the week 7 programming exercises
 2 files changed, 297 insertions(+), 8 deletions(-)
 create mode 100644 week07/week07_assignment_2.ipynb
Counting objects: 5, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 3.66 KiB | 3.66 MiB/s, done.
Total 5 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:paulboal/hds5210-2022.git
   fc2200c..d88bc5b  main -> main
